<a href="https://colab.research.google.com/github/nkcong206/Travel-Recommendation-System/blob/main/database/database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
!pip install -q langchain-google-genai
!pip install -q sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.3 MB/s eta 0:00:00


In [72]:
import psycopg2
import json
import getpass
import os
from sentence_transformers import SentenceTransformer

In [73]:
model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

In [74]:
postgres_url = getpass.getpass("Enter your postgresql url: ")

Enter your postgresql url: ··········


In [75]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


In [76]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [85]:
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

cur.execute("CREATE EXTENSION IF NOT EXISTS postgis;")
cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
conn.commit()

cur.execute("""
DROP SCHEMA IF EXISTS travel_database CASCADE;
CREATE SCHEMA IF NOT EXISTS travel_database;
SET search_path TO travel_database, public;

-- Create a custom type for the Address structure
CREATE TYPE Address AS (
    street TEXT,
    district TEXT,
    city TEXT
);

CREATE TYPE OpeningHours AS (
    open_time TIME WITH TIME ZONE,
    close_time TIME WITH TIME ZONE
);

-- Create the Hotel table with an embedding vector column
CREATE TABLE Hotel (
    hotel_id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    address Address,
    location GEOMETRY(POINT),
    amenities TEXT[ ],
    style TEXT,
    rating DECIMAL(2, 1),
    description TEXT,
    img_url JSON,
    comments JSON
);

-- Create an index on the district of the Address and Rating columns for the Hotel table
-- CREATE INDEX idx_hotel_address_rating ON Hotel(((address).district), rating);

-- Create the Price table for the Hotel table
CREATE TABLE HotelPrice (
    hprice_id SERIAL PRIMARY KEY,
    hotel_id INT REFERENCES Hotel(hotel_id) ON DELETE CASCADE,
    room_type VARCHAR(255),
    capacity INT,
    price INT NOT NULL
);

-- Create an index on the hotel_id for the HotelPrice table
-- CREATE INDEX hotel_idx_address_district ON Hotel(((address).district));
-- CREATE INDEX hotel_idx_rating ON Hotel(rating);
-- CREATE INDEX hotel_idx_location ON Hotel USING GIST(location);

-- Create the TouristAttraction table with an embedding vector column
CREATE TABLE TouristAttraction (
    attraction_id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    address Address,
    location GEOMETRY(POINT),
    attraction_type VARCHAR(255),
    working_hour OpeningHours,
    rating DECIMAL(2, 1),
    tour_duration INTERVAL,
    description TEXT,
    img_url JSON,
    comments JSON
);

-- Create an index on the district of the Address and Rating columns for the TouristAttraction table
-- CREATE INDEX attraction_idx_address_district ON TouristAttraction(((address).district));
-- CREATE INDEX attraction_idx_rating ON TouristAttraction(rating);
-- CREATE INDEX attraction_idx_location ON TouristAttraction USING GIST(location);

-- Create the Price table for the TouristAttraction table
CREATE TABLE AttractionPrice (
    aprice_id SERIAL PRIMARY KEY,
    attraction_id INT REFERENCES TouristAttraction(attraction_id) ON DELETE CASCADE,
    ticket_type VARCHAR(255),
    number_people INT,
    price INT NOT NULL
);

-- Create an index on the attraction_id for the AttractionPrice table
-- CREATE INDEX idx_attractionprice_attractionid ON AttractionPrice(attraction_id);

-- Create the Restaurant table with an embedding vector column
CREATE TABLE Restaurant (
    res_id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    address Address,
    location GEOMETRY(POINT),
    working_hour OpeningHours,
    suitable_for TEXT[],
    restaurant_type TEXT[],
    rating DECIMAL(2, 1),
    description TEXT,
    parking_available BOOLEAN NOT NULL,
    kids_play_area BOOLEAN NOT NULL,
    img_url JSON,
    comments JSON
);

-- Create an index on the district of the Address and Rating columns for the Restaurant table
-- CREATE INDEX res_idx_address_district ON Restaurant(((address).district));
-- CREATE INDEX res_idx_rating ON Restaurant(rating);
-- CREATE INDEX res_idx_location ON Restaurant USING GIST(location);

""")
conn.commit()
cur.close()
conn.close()

Luxury,
Eco-friendly,
Boutique,
Beachfront,
Family-friendly,
Business,
Romantic,
Cultural,
Wellness,
Adventure


In [ ]:
import base64, httpx
from langchain_core.messages import HumanMessage

In [78]:
def get_hotel_styles(data_img):
    image_url = data_img["img_url"][:4]
    image_data = base64.b64encode(httpx.get(image_url[0]).content).decode("utf-8")

    prompt = "Identify the hotel theme from the images. Return only one theme, and it must be one of the following: 'Luxury', 'Eco-friendly', 'Boutique', 'Beachfront', 'Family-friendly', 'Business', 'Romantic', 'Cultural', 'Wellness', or 'Adventure'. Do not provide any other information."

    content=[
            {"type": "text", "text": prompt},
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
            },
        ]

    message = HumanMessage(
        content=content
    )
    respone = llm.invoke([message])
    return respone.content

In [79]:
def get_embedding(text):
    """Trả về embedding của văn bản sử dụng mô hình SentenceTransformer."""
    embedding = model.encode(text)
    return embedding.tolist()


In [86]:
name = 'Khách sạn Hàng Bông'
address = ('Số 71 Hàng Bông', 'Hoàn Kiếm', 'Hà Nội')
location = 'SRID=4326;POINT(105.8514 21.0285)'
rating = 4.5
description = 'Khách sạn Hàng Bông cung cấp dịch vụ chất lượng với vị trí trung tâm thành phố.'
#embedding = get_embedding(description)
img_data = {
  "img_url": [
    "https://ik.imagekit.io/tvlk/apr-asset/Ixf4aptF5N2Qdfmh4fGGYhTN274kJXuNMkUAzpL5HuD9jzSxIGG5kZNhhHY-p7nw/hotel/asset/20050636-b02da2d82245b290beb4e8284828bee4.jpeg?_src=imagekit&tr=c-at_max,f-jpg,fo-auto,h-332,pr-true,q-80,w-480",
    "https://ik.imagekit.io/tvlk/apr-asset/dgXfoyh24ryQLRcGq00cIdKHRmotrWLNlvG-TxlcLxGkiDwaUSggleJNPRgIHCX6/hotel/asset/20050636-cf4c5a5936cc908a23f44ee54f253675.jpeg?_src=imagekit&tr=c-at_max,f-jpg,fo-auto,h-162,pr-true,q-80,w-184",
    "https://ik.imagekit.io/tvlk/apr-asset/dgXfoyh24ryQLRcGq00cIdKHRmotrWLNlvG-TxlcLxGkiDwaUSggleJNPRgIHCX6/hotel/asset/20050636-83c0454f27ab5d93f791429f5b83d9f4.jpeg?_src=imagekit&tr=c-at_max,f-jpg,fo-auto,h-162,pr-true,q-80,w-184",
    "https://ik.imagekit.io/tvlk/apr-asset/dgXfoyh24ryQLRcGq00cIdKHRmotrWLNlvG-TxlcLxGkiDwaUSggleJNPRgIHCX6/hotel/asset/20050636-3e122262768cd4fe06a7a17b3757d032.jpeg?_src=imagekit&tr=c-at_max,f-jpg,fo-auto,h-162,pr-true,q-80,w-184",
    "https://ik.imagekit.io/tvlk/apr-asset/dgXfoyh24ryQLRcGq00cIdKHRmotrWLNlvG-TxlcLxGkiDwaUSggleJNPRgIHCX6/hotel/asset/20050636-3597c1d5421f98dbb8c11e51b237cd86.jpeg?_src=imagekit&tr=c-at_max,f-jpg,fo-auto,h-162,pr-true,q-80,w-184",
    "https://ik.imagekit.io/tvlk/apr-asset/dgXfoyh24ryQLRcGq00cIdKHRmotrWLNlvG-TxlcLxGkiDwaUSggleJNPRgIHCX6/hotel/asset/20050636-bf7ba822dd643b6dab0fac5ff8fcf1e9.jpeg?_src=imagekit&tr=c-at_max,f-jpg,fo-auto,h-162,pr-true,q-80,w-184",
    "https://ik.imagekit.io/tvlk/apr-asset/dgXfoyh24ryQLRcGq00cIdKHRmotrWLNlvG-TxlcLxGkiDwaUSggleJNPRgIHCX6/hotel/asset/20050636-d27cac252faae8e0117924f6bec68a1c.jpeg?_src=imagekit&tr=c-at_max,f-jpg,fo-auto,h-162,pr-true,q-80,w-184"
  ]
}
amenities = [
    "Bồn tắm",
    "Truyền hình cáp",
    "Minibar",
    "Nhân viên xách hành lý",
    "Thức uống chào mừng miễn phí",
    "Dịch vụ giặt ủi",
    "Dịch vụ lưu trữ hành lý",
    "Khu vực hút thuốc",
    "Thang máy",
    "Bữa sáng",
    "Bữa sáng món tự chọn",
    "Mát-xa",
    "Wifi",
    "Đưa đón sân bay"
]
img_url = json.dumps(img_data)
style = get_hotel_styles(img_data)
comments = json.dumps({
    "comments": [
        {"id": 1, "text": "Nhân viên lễ tân rất thân thiện và nhiệt tình hỗ trợ khách hàng. Sảnh chào mát, sạch sẽ và thơm. Phòng ở tiện nghi sạch đẹp. Vị trí khách sạn rất thuận tiện."},
        {"id": 2, "text": "Khá là ok, nhưng ae giặt đồ thì tìm quán nhé"},
        {"id": 3, "text": "Phòng rất đẹp. Mình rất ưng ý. Gia cả hợp ly"}
    ]
})

In [87]:
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()
cur.execute("""SET search_path TO travel_database, public;""")
# Thực hiện câu lệnh INSERT
cur.execute("""
    INSERT INTO Hotel (name, address, location, amenities, style, rating, description, img_url, comments)
    VALUES (%s, ROW(%s, %s, %s), ST_GeomFromText(%s, 4326), %s, %s, %s, %s, %s, %s);
""", (name, address[0], address[1], address[2], location, amenities, style, rating, description, img_url, comments))

# Commit và đóng kết nối
conn.commit()
cur.close()
conn.close()